In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
import tqdm
from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [2]:
_df_gene = pd.read_pickle('gene.pkl')

In [4]:
df = _df_gene.copy()

df['gene_symbol'].value_counts()

Rnf10       2588
Nxn         2524
Dnase1l2    2502
Prkab1      2494
Dbn1        2413
Ap4e1       2147
Bbox1       1317
Ppfibp2     1291
Hsd17b4     1220
Rab39b      1208
            ... 
Drd3           1
Cacna2d2       1
Taar3          1
Rit1           1
Pnkp           1
St8sia6        1
Dhx8           1
Amtn           1
Plxnc1         1
Chd5           1
Name: gene_symbol, Length: 7499, dtype: int64